In [45]:
%%capture imports
!pip install ta
!pip install plotly==5.4
!pip install pandas
!pip install pandas_datareader
!pip install yfinance
!pip install pandoc
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt
from ta.trend import MACD
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [61]:
#tickers = ['DIS', 'NFLX', 'GOOG', 'FB', 'AMZN', 'NVDA', 'TSLA', 'MSFT', 'AAPL']
tickers = ['ETH-USD']
 
now = dt.datetime.now()
start = now - dt.timedelta(days=430)

all_data = {}

def heikin_ashi(df):
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['HKOpen', 'HKHigh', 'HKLow', 'HKClose'])

    #heikin_ashi_df.index = pd.to_datetime(heikin_ashi_df.index).tz_localize('Etc/UCT')

    heikin_ashi_df['HKClose'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['Open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
    heikin_ashi_df['HKHigh'] = heikin_ashi_df.loc[:, ['HKOpen', 'HKClose']].join(df['High']).max(axis=1)
    heikin_ashi_df['HKLow'] = heikin_ashi_df.loc[:, ['HKOpen', 'HKClose']].join(df['Low']).min(axis=1)
    return heikin_ashi_df

for t in tickers:
    #df = fq.get_crypto_data('ETH/USDT', start, now)
    #df = pdr.get_data_yahoo(t, start, now, interval='m')
    df = yf.download(t, 
                      start, 
                      now, progress=False)

    # build complete timeline from start date to end date
    dt_all = pd.date_range(start=df.index[0],end=df.index[-1])
    # retrieve the dates that ARE in the original datset
    dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]
    # define dates with missing values
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

    # add moving averages to df
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA5'] = df['Close'].rolling(window=5).mean()

    # MACD
    macd = MACD(close=df['Close'], 
                window_slow=26,
                window_fast=12, 
                window_sign=9)

    df['MACD'] = macd.macd()
    df['MACD_sig'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()
    ha = heikin_ashi(df)

    df = pd.merge(ha, df, how='outer', left_index=True, right_index=True)
    df = df.dropna()

    # Calculate buy and sell signals for the whole data
    signals = [] 
    signals.append(0)
    in_buy = False
    for index in range(1, df['HKOpen'].size):
        candle = df['HKClose'].iloc[index] - df['HKOpen'].iloc[index]
        candle_last = df['HKClose'].iloc[index - 1] - df['HKOpen'].iloc[index - 1]
        macd = df['MACD_diff'].iloc[index]
        macd_offset = 0.0
        if candle > 0:
            if candle_last < 0 and (macd + macd_offset) < 0:
                signals.append(0)  # MACD fail
            #if candle_last < 0 and in_buy==False:
            #    in_buy = True
            #    signals.append(1)  # Buy
            elif candle_last < 0 and (macd + macd_offset) >= 0 and in_buy==False:
                in_buy = True
                signals.append(1)  # Buy
            else:
                signals.append(0) # HOLD
        else:
            if in_buy == True:
                signals.append(-1) # Sell
                in_buy = False
            else:
                signals.append(0) # No active trade

    df['Sig'] = signals
    all_data[t] = df



In [58]:
all_data['ETH-USD'].tail()

,HKOpen,HKHigh,HKLow,HKClose,Open,High,Low,Close,Adj Close,Volume,MA20,MA5,MACD,MACD_sig,MACD_diff,Sig,Profit
2021-12-23 08:00:00+00:00,3945.33,3945.331675,3897.234375,3918.376404,3913.519043,3933.356445,3897.234375,3929.395752,3929.395752,404746240,3973.757288,3939.389648,-18.429664,-12.375014,-6.054650,0,104.808199
2021-12-23 09:00:00+00:00,3931.85,3935.578125,3910.645264,3928.568787,3932.473633,3935.578125,3910.645264,3935.578125,3935.578125,117009408,3972.651099,3936.193115,-18.701961,-13.640403,-5.061558,0,104.808199
2021-12-23 10:00:00+00:00,3930.21,3958.469727,3917.112793,3938.857910,3933.041504,3958.469727,3917.112793,3946.807617,3946.807617,251154432,3969.957483,3935.794775,-17.806372,-14.473597,-3.332774,0,104.808199
2021-12-23 11:00:00+00:00,3934.53,3948.519287,3934.534662,3945.765808,3945.536865,3948.519287,3942.876709,3946.130371,3946.130371,16869376,3966.793970,3934.220264,-16.955802,-14.970038,-1.985764,0,104.808199
2021-12-23 11:06:02+00:00,3940.15,3948.647217,3940.150235,3948.647217,3948.647217,3948.647217,3948.647217,3948.647217,3948.647217,0,3963.779797,3941.311816,-15.895400,-15.155110,-0.740289,0,104.808199


In [65]:
# Backtest
current_trades = {}
profits = {}
leverage = 4.0
for t in tickers:
    profits[t] = 0.0
    running_pot = []
    pot = 100.0
    running_pot.append(pot)
    df = all_data[t]
    for index in range(1, df['Sig'].size):
        if df['Sig'].iloc[index] >= 1:
            if t in current_trades:
                running_pot.append(pot)
                continue
            else:
                current_trades[t] = df['Close'].iloc[index]
        elif df['Sig'].iloc[index] < 0:
            if t in current_trades:
                profit = ((df['Close'].iloc[index] - current_trades[t]) / current_trades[t]) * (pot * leverage)
                pot += profit
                del current_trades[t]
        running_pot.append(pot)
    df['Profit'] = running_pot

# Show signals
last_candle_index = -2
candle_index = -1
print("Latest Day Candle: ")
print(all_data[tickers[0]].last_valid_index())
print("\nSignals:")
for t in tickers:
    df = all_data[t]
    candle = df['HKClose'].iloc[candle_index] - df['HKOpen'].iloc[candle_index]
    candle_last = df['HKClose'].iloc[last_candle_index] - df['HKOpen'].iloc[last_candle_index]
    macd = df['MACD_diff'].iloc[candle_index] 
    if candle > 0:
        if candle_last < 0 and macd < 0:
            print(t + ": No MACD Confirm")
        elif candle_last < 0 and macd >= 0:
            print(t + ": BUY")
        else:
            print(t + ": HOLD")
    elif candle < 0:
        print(t + ": SELL")

total = 0
for t in tickers:
    df = all_data[t]
    profit = df['Profit'].iloc[-1]
    total += profit 
    print(t + ": " + str(profit))

print ("Total: " + str(total))

Latest Day Candle: 
2021-12-23 00:00:00

Signals:
ETH-USD: SELL
ETH-USD: 24.422590479178165
Total: 24.422590479178165


In [66]:
for t in tickers:
    df = all_data[t]

    # first declare an empty figure
    fig = go.Figure()

    # add subplot properties when initializing fig variable
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True,
                        vertical_spacing=0.01, 
                        row_heights=[0.4,0.3,0.1, 0.4])
    # add OHLC trace on first row
    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['HKOpen'],
                                high=df['HKHigh'],
                                low=df['HKLow'],
                                close=df['HKClose'], 
                                showlegend=False))

    # add moving average traces on same row
    fig.add_trace(go.Scatter(x=df.index, 
                            y=df['MA5'], 
                            opacity=0.7, 
                            line=dict(color='blue', width=2), 
                            name='MA 5'))
    fig.add_trace(go.Scatter(x=df.index, 
                            y=df['MA20'], 
                            opacity=0.7, 
                            line=dict(color='orange', width=2), 
                            name='MA 20'))

    # Plot MACD trace on 2nd row
    fig.add_trace(go.Scatter(x=df.index,
                            y=df['MACD'],
                            line=dict(color='black', width=2)
                            ), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index,
                            y=df['MACD_sig'],
                            line=dict(color='blue', width=1)
                            ), row=2, col=1)

    colors = ['green' if val >= 0 
            else 'red' for val in df['MACD_diff']]

    fig.add_trace(go.Bar(x=df.index, 
                        y=df['MACD_diff'],
                        marker_color=colors
                        ), row=2, col=1)

    colors = ['green' if val >= 0 
            else 'red' for val in df['Sig']]

    fig.add_trace(go.Bar(x=df.index, 
                        y=df['Sig'],
                        marker_color=colors
                        ), row=3, col=1)

    fig.add_trace(go.Scatter(x=df.index, 
                            y=df['Profit'], 
                            opacity=0.7, 
                            line=dict(color='green', width=2), 
                            name='Profit'), row=4, col=1)
    # Plot volume trace on 3rd row
    #colors = ['green' if row['HKClose'] - row['HKOpen'] >= 0 
    #        else 'red' for index, row in df.iterrows()]
    #fig.add_trace(go.Bar(x=df.index, 
    #                    y=df['Volume'],
    #                    marker_color=colors
    #                    ), row=4, col=1)

    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", showgrid=False, row=2, col=1)
    fig.update_yaxes(title_text="Signal", showgrid=False, row=3, col=1)
    fig.update_yaxes(title_text="Profit", showgrid=False, row=4, col=1)
    #fig.update_yaxes(title_text="Volume", showgrid=False, row=4, col=1)

    # update layout by changing the plot size, hiding legends & rangeslider, and removing gaps between dates
    fig.update_layout(height=450, width=1200, 
                    showlegend=False, 
                    title=t,
                    xaxis_rangeslider_visible=False,
                    xaxis_rangebreaks=[dict(values=dt_breaks)])

    # removing white space
    fig.update_layout(margin=go.layout.Margin(
            l=20, #left margin
            r=20, #right margin
            b=20, #bottom margin
            t=40  #top margin
        ))

    #from IPython.display import Image
    #image_bytes = fig.to_image(format='png', width=1200, height=700, scale=1)
    #display(Image(image_bytes))

    fig.show()

